# small course on the inverse theory

In [1]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")

  Activating project at `~/Documents/Github/flexOPT`


In [2]:
using Symbolics
include("../src/commonBatchs.jl")
using .commonBatchs

problem setting

In [25]:
# dimensions
modelDimension = 2
dataDimension  = 3

# models (nₑ distribution in the Earth)
@variables m[1:modelDimension] # a model
@variables mTrue[1:modelDimension] # the ground truth
@variables m⁰[1:modelDimension] # our initial guess (PREM or 3D initial model)
@variables δm[1:modelDimension] # an arbitrary model perturbation from m₀ : m = m₀+δm
@variables Δm[1:modelDimension]
Δm =[mTrue[i] - m⁰[i] for i ∈ 1:modelDimension] # model perturbation required to have the goround truth
#δm =[m[i] - m⁰[i] for i ∈ 1:modelDimension]
# data (predicted and observed), since we mimic 'symbolic partial derivations' by multiplications, u=G(m) stays only in our heart
# maybe WHATEVER!!!
@variables u(m...)[1:dataDimension]
@variables d[1:dataDimension] 
#@variables u⁰(m⁰)[1:dataDimension]
#@variables u[1:dataDimension] # G(m)
#@variables d[1:dataDimension] # G(mTrue)
#@variables u⁰[1:dataDimension] # G(m₀)

Δd=[d[i] - u⁰[i] for i ∈ 1:dataDimension]# initial data residual (I prefer this sign in order to be consistent with Δm definition)
δd = [d[i] - u[i] for i ∈ 1:dataDimension] # data residual for synthetic data predicted for an arbitrary model m = m₀+δm

# partials (in a wild symbolic way), but all the ∇s are linear so let's be super super simple (I don't really use the functionalities of )

#@variables ∂m[1:modelDimension]
#@variables ∂δm[1:modelDimension]
#@variables ∂m⁰[1:modelDimension]
∂m = [Differential(m[i]) for i ∈ 1:modelDimension]
∂m⁰ = [Differential(m⁰[i]) for i ∈ 1:modelDimension]
∂δm = [Differential(δm[i]) for i ∈ 1:modelDimension]

2-element Vector{Differential}:
 Differential(δm[1])
 Differential(δm[2])

In [18]:
#cost function for F(m)
F = expand(1//2*sum(δd[i]^2 for i in 1:dataDimension))

(1//2)*(d[1]^2) - d[1]*(u(m[1], m[2]))[1] + (1//2)*(d[2]^2) - d[2]*(u(m[1], m[2]))[2] + (1//2)*(d[3]^2) - d[3]*(u(m[1], m[2]))[3] + (1//2)*((u(m[1], m[2]))[1]^2) + (1//2)*((u(m[1], m[2]))[2]^2) + (1//2)*((u(m[1], m[2]))[3]^2)

Let's develop a Taylor expansion 

In [19]:
# first derivative = gradient direction
∇F = [expand_derivatives(∂m[i](F)) for i ∈ 1:modelDimension] # sorry that ∂ can be found after the mathematical expressions but here it's treated as variables and it's ok to manipulate manually

2-element Vector{Num}:
 -d[1]*Differential(m[1])((u(m[1], m[2]))[1]) - d[2]*Differential(m[1])((u(m[1], m[2]))[2]) - d[3]*Differential(m[1])((u(m[1], m[2]))[3]) + (u(m[1], m[2]))[1]*Differential(m[1])((u(m[1], m[2]))[1]) + (u(m[1], m[2]))[2]*Differential(m[1])((u(m[1], m[2]))[2]) + (u(m[1], m[2]))[3]*Differential(m[1])((u(m[1], m[2]))[3])
 -d[1]*Differential(m[2])((u(m[1], m[2]))[1]) - d[2]*Differential(m[2])((u(m[1], m[2]))[2]) - d[3]*Differential(m[2])((u(m[1], m[2]))[3]) + (u(m[1], m[2]))[1]*Differential(m[2])((u(m[1], m[2]))[1]) + (u(m[1], m[2]))[2]*Differential(m[2])((u(m[1], m[2]))[2]) + (u(m[1], m[2]))[3]*Differential(m[2])((u(m[1], m[2]))[3])

In [20]:
∇∇F = [expand_derivatives(∂m[j](∇F[i])) for i ∈ 1:modelDimension, j ∈ 1:modelDimension ] # second derivatives

2×2 Matrix{Num}:
                                                                                                                -d[1]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[1])) - d[2]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[2])) - d[3]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[3])) + (u(m[1], m[2]))[1]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[1])) + (u(m[1], m[2]))[2]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[2])) + (u(m[1], m[2]))[3]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[3])) + Differential(m[1])((u(m[1], m[2]))[1])^2 + Differential(m[1])((u(m[1], m[2]))[2])^2 + Differential(m[1])((u(m[1], m[2]))[3])^2  …  -d[1]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[1])) - d[2]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[2])) - d[3]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[3])) + (u(m[1], m[2]))[1]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[1])) + (u(m[1], m[2]))[2]*Dif

In [21]:

# if you don't do this, just comment it to see what's happening at the end
ignore_second_derivatives = Dict(∂m[i](∂m[j](u[k]))=>0 for i ∈ 1:modelDimension, j ∈ 1:modelDimension, k ∈ 1:dataDimension)

Dict{Num, Int64} with 12 entries:
  Differential(m[1])(Differential(m[2])((u(m[1], m[2]))[1])) => 0
  Differential(m[2])(Differential(m[2])((u(m[1], m[2]))[1])) => 0
  Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[1])) => 0
  Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[1])) => 0
  Differential(m[1])(Differential(m[2])((u(m[1], m[2]))[2])) => 0
  Differential(m[2])(Differential(m[2])((u(m[1], m[2]))[2])) => 0
  Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[3])) => 0
  Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[3])) => 0
  Differential(m[1])(Differential(m[2])((u(m[1], m[2]))[3])) => 0
  Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[2])) => 0
  Differential(m[2])(Differential(m[2])((u(m[1], m[2]))[3])) => 0
  Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[2])) => 0

In [22]:
∇∇F = mySimplify(substitute(∇∇F, ignore_second_derivatives))

2×2 Matrix{SymbolicUtils.BasicSymbolic{Real}}:
 Differential(m[1])((u(m[1], m[2]))[1])^2 + Differential(m[1])((u(m[1], m[2]))[2])^2 + Differential(m[1])((u(m[1], m[2]))[3])^2                                                                                                                 …  Differential(m[1])((u(m[1], m[2]))[1])*Differential(m[2])((u(m[1], m[2]))[1]) + Differential(m[1])((u(m[1], m[2]))[2])*Differential(m[2])((u(m[1], m[2]))[2]) + Differential(m[1])((u(m[1], m[2]))[3])*Differential(m[2])((u(m[1], m[2]))[3])
 Differential(m[1])((u(m[1], m[2]))[1])*Differential(m[2])((u(m[1], m[2]))[1]) + Differential(m[1])((u(m[1], m[2]))[2])*Differential(m[2])((u(m[1], m[2]))[2]) + Differential(m[1])((u(m[1], m[2]))[3])*Differential(m[2])((u(m[1], m[2]))[3])     Differential(m[2])((u(m[1], m[2]))[1])^2 + Differential(m[2])((u(m[1], m[2]))[2])^2 + Differential(m[2])((u(m[1], m[2]))[3])^2

# substitution for $m^0$

In [ ]:

dict_for_∂m⁰ = Dict(∂m[i] => ∂m⁰[i] for i ∈ 1:modelDimension)
dict_for_m⁰= merge(dict_for_m⁰,dict_for_∂m⁰)

Dict{Any, Any} with 5 entries:
  (u(m[1], m[2]))[2] => (u⁰(m⁰))[2]
  Differential(m[2]) => Differential(m⁰[2])
  (u(m[1], m[2]))[1] => (u⁰(m⁰))[1]
  (u(m[1], m[2]))[3] => (u⁰(m⁰))[3]
  Differential(m[1]) => Differential(m⁰[1])

In [24]:
F⁰= substitute(F,dict_for_m⁰)

(1//2)*(d[1]^2) - d[1]*(u⁰(m⁰))[1] + (1//2)*(d[2]^2) - d[2]*(u⁰(m⁰))[2] + (1//2)*(d[3]^2) - d[3]*(u⁰(m⁰))[3] + (1//2)*((u⁰(m⁰))[1]^2) + (1//2)*((u⁰(m⁰))[2]^2) + (1//2)*((u⁰(m⁰))[3]^2)

In [ ]:

∇F⁰= substitute(∇F,dict_for_m⁰) 
∇∇F⁰= substitute(∇∇F,dict_for_m⁰) 

In [ ]:
# we want to develop a Taylor expansion 
F_Taylor⁰ = F⁰ + expand(∇F⁰'*δm) + expand(1//2*δm'*∇∇F⁰*δm) # and δm^3 -> ignored

We are just looking for $\delta m$ which can make $F^0$ has its plateau
\begin{equation}
\nabla  F = 0
\end{equation}

In [ ]:
∇F_Taylor⁰ = [expand_derivatives(expand(∂m[i](F_Taylor⁰))) for i ∈ 1:modelDimension]

In [ ]:
# which terms are 0?
dict_data_is_model_independent = Dict(∂m⁰[j]*d[i] =>0  for i ∈ 1:dataDimension, j ∈ 1:modelDimension)
dict_data_is_model_independent = merge(dict_data_is_model_independent,Dict(∂δm[j]*d[i] =>0  for i ∈ 1:dataDimension, j ∈ 1:modelDimension))

∇F_Taylor⁰ = substitute(∇F_Taylor⁰,dict_data_is_model_independent)

In [ ]:

dict_I_need_to_do_this_manually = Dict(∂δm[j]*δm[j]^2 => 2*δm[j] for j ∈ 1:modelDimension)
dict_I_need_to_do_this_manually = merge(dict_I_need_to_do_this_manually,Dict(∂δm[j]*δm => 1 for j ∈ 1:modelDimension ))
tmpDict = merge(dict_data_is_model_independent,dict_I_need_to_do_this_manually)

In [ ]:
mySimplify.(substitute(∇F_Taylor⁰,tmpDict))

problem setting (the way I wanted to do but still some bugs in Symbolics.jl)

In [ ]:
# dimensions
modelDimension = 3
dataDimension  = 5

# models (nₑ distribution in the Earth)
@variables m[1:modelDimension] # a model
@variables mTrue[1:modelDimension] # the ground truth

# forward modelling operator 
@variables G(..)   # G(i, m) is a symbolic function of m


# data (predicted and observed)
u = [G(i, m) for i in 1:dataDimension] # u is the function of m
d = [G(i, mTrue) for i in 1:dataDimension] # d is not the function of m


# initial data misfit
Δd = u .- d



In [ ]:
∇ = Differential(m[1:modelDimension])
@show ∇(u)

In [ ]:
# cost function
F=mySimplify(1//2*sum(expand(Δd[i]^2) for i in 1:dataDimension))

In [ ]:
example=mySimplify(Differential(m[1])F)

In [ ]:
dict = Dict([Differential(m[1])mTrue => 0])

In [ ]:
substitute(example,dict)

In [ ]:
# the Δm that we need# symbolic input vector
@variables Δm[1:modelDimension]

In [ ]:
sum(gradientDirection[i] * Δm[i] for i in 1:modelDimension)